# COVID-19 in Denmark and the world prediction with SEIR-D model

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os
from tqdm.notebook import tqdm
from scipy.integrate import solve_ivp
import numpy
import datetime
from datetime import timedelta
from matplotlib import dates
import plotly.graph_objects as go
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_log_error, mean_squared_error
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.offline as ply
ply.init_notebook_mode(connected=True)

#%matplotlib inline

Thanks to 
* anjum48 https://www.kaggle.com/anjum48/seir-hcd-model for a great kernel that served as inspiration for this study

In [2]:
train = pd.read_csv('../input/novel-corona-virus-2019-dataset/covid_19_data.csv') 
train['Date_datetime'] = train['ObservationDate'].apply(lambda x: (datetime.datetime.strptime(x, "%m/%d/%Y"))) 
pop_info = pd.read_csv('/kaggle/input/covid19-population-data/population_data.csv') 

 ### Data Cleaning

In [3]:
train.tail(5)

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Date_datetime
48090,48091,06/22/2020,Zacatecas,Mexico,2020-06-23 04:33:22,706.0,75.0,442.0,2020-06-22
48091,48092,06/22/2020,Zakarpattia Oblast,Ukraine,2020-06-23 04:33:22,2188.0,57.0,842.0,2020-06-22
48092,48093,06/22/2020,Zaporizhia Oblast,Ukraine,2020-06-23 04:33:22,555.0,16.0,391.0,2020-06-22
48093,48094,06/22/2020,Zhejiang,Mainland China,2020-06-23 04:33:22,1269.0,1.0,1267.0,2020-06-22
48094,48095,06/22/2020,Zhytomyr Oblast,Ukraine,2020-06-23 04:33:22,1270.0,23.0,633.0,2020-06-22


In [4]:
train = train.rename(columns={'Country/Region': 'Country_Region', 'Confirmed': 'ConfirmedCases', 'Deaths': 'Fatalities', 'Province/State': 'Province_State'})


In [5]:
train = train.query("Province_State != 'Faroe Islands'")
train = train.query("Province_State != 'Greenland'")
train = train.query('ConfirmedCases > 1').copy() 


* # Visualization of dataset

In [6]:
denmark = train[train['Country_Region']=='Denmark']
fig = go.Figure()
fig.update_layout(width=800, height=500)
fig.add_trace(go.Scatter(x=denmark['Date_datetime'], 
                         y=denmark['ConfirmedCases'],name="Confirmed Cases"))
fig.add_trace(go.Scatter(x=denmark['Date_datetime'], 
                         y=denmark['Fatalities'],name="Fatalities"))
fig.update_layout(template='seaborn',title="Dataset")
fig.show()

# SEIR-D

In [7]:
def seir_d(t, y, N, beta, rho, alpha, theta): 
    S, E, I, R, D = y
    if callable(beta) == True :
        dSdt = -beta(t) * S * I 
        dEdt = beta(t) * S * I  - alpha * E
    else : 
        dSdt = -beta * S * I 
        dEdt = beta  * S * I - alpha * E
    dIdt = alpha * E -(rho+theta/(1-theta)*rho) * I
    dRdt = rho * I
    dDdt = theta / (1-theta) * rho * I
    
    return dSdt, dEdt, dIdt, dRdt, dDdt

In [8]:
def model_decay(params, data, population, return_solution=False, full_validation=True, forecast_days=0):
    R_0, theta, k, L, R_t, D_rec, D_inc = params # R_0, fatality rate, k form, L shape, R end
    N = population # Population of each country
    
    n_infected = data['ConfirmedCases'].iloc[0] 
    n_recovered = data['Recovered'].iloc[0]
    n_dead = data['Fatalities'].iloc[0]
    max_days = len(data) + forecast_days 
    rho = 1.0 / D_rec
    alpha = 1.0 / D_inc    
    
    y0 = N - n_infected, 0, n_infected, 0 ,0  

    def beta(t):
        return (((R_0-R_t) / (1 + (t/L)**k)+R_t) * rho ) / N

    t = np.arange(max_days)

    # Solve the SEIR differential equation.
    sol = solve_ivp(seir_d, [0, max_days],y0,  args=(N, beta, rho, alpha, theta),t_eval=t)
    sus, exp, inf, rec, dead = sol.y    
        
    # Predict confirmedcases
    y_pred_cases = inf + rec + dead
    y_true_cases = data['ConfirmedCases'].values
    
    # Predict fatalities
    y_pred_dead = dead
    y_true_dead = data['Fatalities'].values    
 
    
    optim_days = 20 # Days to optimise for

    #weights = 1 / np.arange(1, optim_days+1)[::-1]  # Recent data is more heavily weighted
    #weights = np.linspace(0,1,20)
    # using mean squre log error to evaluate
    msle_conf = mean_squared_log_error(y_true_cases[-optim_days:], y_pred_cases[-optim_days:])
    msle_dead = mean_squared_log_error(y_true_dead[-optim_days:], y_pred_dead[-optim_days:])
    if full_validation == True :
        msle = np.mean([msle_conf,msle_dead])
    else : 
        msle = msle_conf
        
    if return_solution:
        return sol
    else:
        return msle

In [9]:
# Use a constant reproduction number
def model_const(params, data, population, return_solution=False, full_validation=True, forecast_days=0):
    R_0_start, theta, D_rec, D_inc = params # Paramaters, R0 and alpha 
    N = population # Population of each country
    
    n_infected = data['ConfirmedCases'].iloc[0] 
    n_recovered = data['Recovered'].iloc[0]
    n_dead = data['Fatalities'].iloc[0]    
    max_days = len(data) + forecast_days # How many days want to predict
    rho = 1.0 / D_rec
    alpha = 1.0 / D_inc    
    beta = (R_0_start * rho)/N   # R_0 = beta / Rho
    
    S0, E0, I0, R0, D0 = N - n_infected, 0, n_infected, 0 ,0  
    y0 = S0, E0, I0, R0, D0 # Initial conditions vector

    t = np.arange(max_days)

    # Solve the SEIR differential equation.
    sol = solve_ivp(seir_d, [0, max_days],y0,  args=(N, beta, rho, alpha, theta),t_eval=t)
    sus, exp, inf, rec, dead = sol.y    
        
    # Predict confirmedcases
    y_pred_cases = inf + rec + dead
    y_true_cases = data['ConfirmedCases'].values 

    # Predict fatalities
    y_pred_dead = dead
    y_true_dead = data['Fatalities'].values    
    
    optim_days = 20  # Days to optimise for finds the lowest num. 

    weights = 1 / np.arange(1, optim_days+1)[::-1]  # Recent data is more heavily weighted
    # using mean squre log error to evaluate
        
    msle_conf = mean_squared_log_error(y_true_cases[-optim_days:], y_pred_cases[-optim_days:])
    msle_dead = mean_squared_log_error(y_true_dead[-optim_days:], y_pred_dead[-optim_days:])
    
    if full_validation == True :
        msle = np.mean([msle_conf,msle_dead])
    else : 
        msle = msle_conf

    if return_solution:
        return sol
    else:
        return msle

where L is a description of the rate of decay, either the time until half decay or the time until full decay (see above), and k is a shape parameter (no dimension).

In [10]:
def fit_seir_model(data, country, forecast_days = 14, make_plot = True, validation = True, full_validation = True, decay_mode = None):
    country_data = data[data['Country_Region'] == country]
    country_data = country_data.query('ConfirmedCases > 0').copy() 
        
    # Train - test split
    x = 21 ## Validation days.
    if validation : # 
        valid_data = country_data[-x:].copy() ##
        train_data = country_data[:-x].copy() ## 
    else : 
        valid_data = country_data.copy() ## 
        train_data = country_data.copy() ## k
    
    # 
    try:
        population = pop_info[pop_info['Name']==country]['Population'].tolist()[0]
    except IndexError:
        print ('country not in population set, '+str(country))
            
    n_infected = train_data['ConfirmedCases'].iloc[0]
        
    # 
    
    #  R_0, theta, D_rec, D_inc
    model_const_res = minimize(model_const, [1.8, 0.04,15,4.9], 
                        bounds=((0, 6.49), (0.01, 0.15),(3.48,18),(4.9,5.9)),
                        args=(train_data, population, False,full_validation),
                        method='L-BFGS-B')
    
    # R_0, fatality rate, k form, L shape, Rt end, D_rec, D_inc
    model_decay_res = minimize(model_decay, [1.8, 0.04, 2, 20, 1 ,15, 4.9], 
                    bounds=((0, 6.49), (0.01, 0.15), (0, 5), (0, 200),(0,6.49),(3.48,18),(4.9,5.9)),
                    args=(train_data, population, False,full_validation),
                    method='L-BFGS-B')
    

    test_end = train['Date_datetime'].max() 
    train_min = train_data.Date_datetime.min()
    train_max = train_data.Date_datetime.max()
    test_period =(test_end - train_max).days 
    
    dates_all = pd.to_datetime(np.arange(train_min, test_end+timedelta(1)+timedelta(forecast_days), dtype='datetime64[D]'))
    
    
    print(f'msle: model_const_res {model_const_res.fun:0.5f}  model_decay_res {model_decay_res.fun:0.5f}')

    if model_decay_res.fun < model_const_res.fun :
        sol = model_decay(model_decay_res.x, train_data, population, True,full_validation, test_period+forecast_days)
        res = model_decay_res
        R_0,alpha,k,L,R_t, D_rec, D_inc = res.x
        sus, exp, inf, rec, dead = sol.y
   
        t = np.arange(len(dates_all))
        R_t = pd.Series((R_0-R_t) / (1 + (t/L)**k) + R_t, dates_all)
        R_t_median = pd.Series(np.median(R_t), dates_all)
        print(f'R_0_start {res.x[0]:0.5f} theta {res.x[1]:0.5f} k {res.x[2]:0.5f} L {res.x[3]:0.5f} R_t_end {res.x[4]:0.5f} D_rec {res.x[5]:0.5f} D_inc {res.x[6]:0.5f}')
        if validation : 
            valid_msle = model_decay_res.fun
        else :
            valid_msle = 0
    else :
        sol = model_const(model_const_res.x, train_data, population, True,full_validation, test_period+forecast_days)
        res = model_const_res
        R_0_start, alpha, D_rec, D_inc = res.x
        sus, exp, inf, rec, dead = sol.y
      
        t = np.arange(len(dates_all))
        R_t = pd.Series([model_const_res.x[0]] * len(dates_all), dates_all)
        print(f'using constant r0 {res.x[0]:0.5f} theta {res.x[1]:0.5f} D_rec {res.x[2]:0.5f} D_inc {res.x[3]:0.5f}')
        if validation : 
            valid_msle = model_decay_res.fun
        else :
            valid_msle = 0


    y_pred = pd.DataFrame({
        'ConfirmedCases': np.diff((inf + (rec + dead) ),prepend=0).cumsum(),
        'Recovered': np.around(rec,0),
        'Fatalities': np.around(dead,0),
        'R': R_t,
    })
    
 
    y_pred_valid = y_pred.iloc[len(train_data):len(train_data)+len(valid_data)]

    y_true_valid = valid_data[['ConfirmedCases']]
    y_true_valid3 = valid_data[['Fatalities']]

    msle_val_conf = mean_squared_log_error(y_true_valid['ConfirmedCases'].values, y_pred_valid['ConfirmedCases'][:len(train_data)].values)
    msle_val_death = mean_squared_log_error(y_true_valid3['Fatalities'].values, y_pred_valid['Fatalities'][:len(train_data)].values)
    msle_val = np.mean([msle_val_conf,msle_val_death])
    print('msle_validation ' + str(msle_val))
    



    if make_plot:
        print(f'Validation MSLE: {valid_msle:0.5f}, Median R over perioden: {R_t.median():0.5f}, Raten for fatale tilfælde: {res.x[1]:0.5f} ')
        
        fig = make_subplots(specs=[[{"secondary_y": True}]])
        
        fig.add_trace(
            go.Scatter(x=dates_all, y=np.around(exp,0),line_color="coral", name="Eksponerede"),
            secondary_y=False,
        )
        fig.add_trace(
            go.Scatter(x=dates_all, y=np.around(inf,0),line_color="indianred", name="Infektiøse"),
            secondary_y=False,
        )
        fig.add_trace(
            go.Scatter(x=dates_all, y=np.around(rec,0),line_color="mediumseagreen", name="Raske"),
            secondary_y=False,
        )
        fig.add_trace(
            go.Scatter(x=dates_all, y=np.around(dead,0), name="Døde",line=dict(color='black', width=2)),
            secondary_y=False,
        )
        fig.add_trace(
            go.Scatter(x=dates_all, y=R_t, name="R",line=dict(color='cyan', width=2, dash='dash')),
            secondary_y=True,
        )
                
        fig.add_trace(
            go.Scatter(x=dates_all, y=R_t_median, name="R Median",line=dict(color='purple', width=2, dash='dash')),
            secondary_y=True,
        )
        
        fig.update_layout(
            title_text="<b>SEIR-D</b> model i "+ str(country) +' med ' + str(forecast_days) + ' dages forudsigelse'
        )

        # Set x-axis title
        fig.update_xaxes(title_text="Datoer")

        # Set y-axes titles
        fig.update_yaxes(title_text="<b>Population</b>", secondary_y=False)
        fig.update_yaxes(title_text="<b>Effektiv</b> R", secondary_y=True)
        
        fig.update_layout(hovermode="x")

        fig.show()
        
        fig = go.Figure()
        fig.add_trace(
            go.Scatter(x=train_data['Date_datetime'], y=np.around(train_data['ConfirmedCases'],0), name="Data C",line=dict(color='indianred', width=2)),
        )
        fig.add_trace(
            go.Scatter(x=dates_all, y=np.around(y_pred['ConfirmedCases'],0), name="Model C",line=dict(color='coral', width=2)),
        )
        
        fig.add_trace(
            go.Scatter(x=train_data['Date_datetime'], y=np.around(train_data['Fatalities'],0), name="Data D",line=dict(color='black', width=2)),
        )
        
        fig.add_trace(
            go.Scatter(x=dates_all, y=np.around(y_pred['Fatalities'],0), name="Model D",line=dict(color='Gray', width=2)),
        )

        if validation : 
            fig.add_trace(
            go.Scatter(x=valid_data['Date_datetime'], y=np.around(y_true_valid['ConfirmedCases'],0), name="Data C (valid)",line=dict(color='indianred', width=2,dash='dot')),
        )
            fig.add_trace(
        go.Scatter(x=valid_data['Date_datetime'], y=np.around(y_pred_valid['ConfirmedCases'][:len(train_data)],0), name="Model C(valid)",line=dict(color='coral', width=2,dash='dot')),
        )
        # fig.add_trace(
        # go.Scatter(x=valid_data['Date_datetime'], y=y_true_valid2['Recovered'], name="Data R (valid)"),
        #   )
        #fig.add_trace(
        #go.Scatter(x=valid_data['Date_datetime'], y=y_pred_valid['Recovered'][:len(train_data)], name="Model R (valid)"),
        #)
            fig.add_trace(
            go.Scatter(x=valid_data['Date_datetime'], y=np.around(y_true_valid3['Fatalities'],0), name="Data D (valid)",line=dict(color='black', width=2,dash='dot')),
        )
            fig.add_trace(
        go.Scatter(x=valid_data['Date_datetime'], y=np.around(y_pred_valid['Fatalities'][:len(train_data)],0), name="Model D (valid)",line=dict(color='Gray', width=2,dash='dot')),
        )
        
        # Add figure title
        fig.update_layout(
            title_text="<b>Model</b> sammenlignet med træningsset" +' med ' + str(forecast_days) + ' dages forudsigelse'
        )
        
        # Set x-axis title
        fig.update_xaxes(title_text="Datoer")

        # Set y-axes titles
        fig.update_yaxes(title_text="<b>Population</b>")
        
        fig.update_layout(hovermode="x")
            
        fig.show()
    
            

# Machine learning

# Scandinavian countries

In [11]:
fit_seir_model(train,'Denmark',forecast_days=0 ,make_plot=True,validation=True,full_validation=True)

msle: model_const_res 0.24566  model_decay_res 0.00007
R_0_start 6.48552 theta 0.05114 k 4.92001 L 22.78062 R_t_end 0.55139 D_rec 6.27516 D_inc 4.90166
msle_validation 0.0005890034871168844
Validation MSLE: 0.00007, Median R over perioden: 0.61580, Raten for fatale tilfælde: 0.05114 


In [12]:
italy = train[train['Country_Region']=='Italy'].groupby('Date_datetime').sum().copy().reset_index()
italy['Country_Region']='Italy'
fit_seir_model(italy,'Italy',forecast_days=0,make_plot=True,validation=True,full_validation=True)

msle: model_const_res 0.31299  model_decay_res 0.00001
R_0_start 6.48998 theta 0.14371 k 5.00000 L 24.38330 R_t_end 0.88709 D_rec 3.48000 D_inc 4.90000
msle_validation 4.84362234952443e-05
Validation MSLE: 0.00001, Median R over perioden: 0.91283, Raten for fatale tilfælde: 0.14371 


In [13]:
iran = train[train['Country_Region']=='Iran'].groupby('Date_datetime').sum().copy().reset_index()
iran['Country_Region']='Iran'
fit_seir_model(iran,'Iran',forecast_days=300,make_plot=True,validation=True,full_validation=True)

msle: model_const_res 0.33936  model_decay_res 0.00078
R_0_start 6.49000 theta 0.05768 k 5.00000 L 21.27750 R_t_end 0.97501 D_rec 3.63539 D_inc 4.94042
msle_validation 0.009453545761431457
Validation MSLE: 0.00078, Median R over perioden: 0.97507, Raten for fatale tilfælde: 0.05768 


In [14]:
sweden = train[train['Country_Region']=='Sweden'].groupby('Date_datetime').sum().copy().reset_index()
sweden['Country_Region']='Sweden'
fit_seir_model(sweden,'Sweden',forecast_days=50,make_plot=True,validation=True,full_validation=True)

msle: model_const_res 0.27716  model_decay_res 0.00022
R_0_start 5.92044 theta 0.12757 k 4.92272 L 22.62532 R_t_end 1.05596 D_rec 4.44247 D_inc 4.91241
msle_validation 0.0181274769660724
Validation MSLE: 0.00022, Median R over perioden: 1.06381, Raten for fatale tilfælde: 0.12757 


In [15]:
fit_seir_model(train,'Norway',forecast_days=0,make_plot=True,validation=True,full_validation=True)

msle: model_const_res 0.18579  model_decay_res 0.00003
R_0_start 6.48999 theta 0.02852 k 3.89428 L 22.84609 R_t_end 0.14559 D_rec 7.49057 D_inc 4.90000
msle_validation 0.0005502896469379371
Validation MSLE: 0.00003, Median R over perioden: 0.31536, Raten for fatale tilfælde: 0.02852 


In [16]:
fit_seir_model(train,'Finland',forecast_days=0,make_plot=True,validation=True,full_validation=True)

msle: model_const_res 0.20992  model_decay_res 0.00008
R_0_start 6.47505 theta 0.04881 k 5.00000 L 22.90462 R_t_end 0.65025 D_rec 6.86670 D_inc 4.90270
msle_validation 0.0005635015090509866
Validation MSLE: 0.00008, Median R over perioden: 0.70342, Raten for fatale tilfælde: 0.04881 
